In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

In [2]:
class CLoss(keras.losses.Loss):
    def __init__(self, threshold = 0.1, **kwargs):
        super().__init__(**kwargs)
        self.threshold = tf.constant(threshold, dtype = tf.float32)
    
    def call(self, y_true, y_pred):
        y_true = tf.constant(y_true, dtype = tf.float32)
        y_pred = tf.constant(y_pred, dtype = tf.float32)
        error = tf.abs(tf.subtract(y_true, y_pred))
        error = error[tf.greater(error, self.threshold)]
        return tf.divide(tf.reduce_sum(tf.subtract(error, self.threshold)), y_pred.shape[0])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold.numpy()}

In [3]:
closs = CLoss()
closs.get_config()

{'reduction': 'auto', 'name': None, 'threshold': 0.1}

In [4]:
closs.call([1, 2, 3, 4, 5], [1.5, 3, 2, 4, 5])

<tf.Tensor: shape=(), dtype=float32, numpy=0.43999997>

In [5]:
class CMetric(keras.metrics.Metric):
    def __init__(self, threshold = 0.1, **kwargs):
        super().__init__(**kwargs)
        self.threshold = tf.constant(threshold, dtype = tf.float32)
        self.error = self.add_weight('error', initializer = 'zeros')
        self.count = self.add_weight('count', initializer = 'zeros')
    
    def update_state(self, y_true, y_pred):
        y_true = tf.constant(y_true, dtype = tf.float32)
        y_pred = tf.constant(y_pred, dtype = tf.float32)
        
        error = tf.abs(tf.subtract(y_true, y_pred))
        error = tf.subtract(error[tf.greater(error, self.threshold)], self.threshold)
        
        self.error.assign_add(tf.reduce_sum(error))
        self.count.assign_add(y_pred.shape[0])
    
    def result(self):
        return tf.divide(self.error, self.count)
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'threshold': self.threshold.numpy()}

In [6]:
cmetric = CMetric()

In [7]:
cmetric.get_config()

{'name': 'c_metric', 'dtype': 'float32', 'threshold': 0.1}

In [8]:
cmetric.update_state([1,], [1.5])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.4>

In [9]:
cmetric.update_state([2,], [3,])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.65>

In [10]:
cmetric.update_state([3,], [2,])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.7333333>

In [11]:
cmetric.update_state([4, 5], [4, 5])
cmetric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.43999997>

In [12]:
cmetric.reset_states()

In [13]:
class CLayer(keras.layers.Layer):
    def __init__(self, units, activation, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name = 'kernel', shape = [batch_input_shape[-1], self.units],
            initializer = 'glorot_normal')
        self.bias = self.add_weight(
            name = 'bias', shape = [self.units],
            initializer = 'zeros')
    
    def call(self, X):
        return self.activation(X@self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list[:-1]+[self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, 'units': self.units, 'activation': self.activation.__name__}

In [14]:
clayer = CLayer(3, 'relu')

In [15]:
clayer.get_config()

{'name': 'c_layer',
 'trainable': True,
 'dtype': 'float32',
 'units': 3,
 'activation': 'relu'}

In [16]:
clayer.build([2])

In [17]:
clayer.call([[1, 1], [0, 0]])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [18]:
clayer.weights

[<tf.Variable 'kernel:0' shape=(2, 3) dtype=float32, numpy=
 array([[-0.54496485,  0.22378716, -1.0217781 ],
        [-0.6369676 , -0.446637  , -0.38424966]], dtype=float32)>,
 <tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]

In [19]:
clayer.bias

<tf.Variable 'bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

In [20]:
model1 = keras.models.Sequential([
    keras.layers.Input(shape = 2),
    keras.layers.Dense(units = 3, activation = 'relu'),
    keras.layers.Dense(units = 1, activation = 'sigmoid')
])

In [21]:
model1.compile(optimizer = 'sgd', loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [22]:
model1.fit(np.array([[1, 1], [2, 2], [3, 3], [4, 4], [5, 5]]), np.array([1, 1, 0, 0, 0]),
          validation_split = 0.1, epochs = 10)

Train on 4 samples, validate on 1 samples
Epoch 1/10
4/4 [==============================] - 1s 280ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931

In [23]:
model1.predict(np.array([[1, 2]]))

array([[0.5426884]], dtype=float32)

In [24]:
model1.evaluate(np.array([[1, 1]]), np.array([1]))

1/1 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.0000e+00


[0.6931471824645996, 0.0]

In [25]:
model2 = keras.models.Sequential([
    keras.layers.Input(shape = 2),
    CLayer(units = 3, activation = 'relu'),
    keras.layers.Dense(units = 1, activation = 'sigmoid')
])

In [26]:
model2.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [27]:
model2.fit(np.array([[1, 1], [2, 2], [3, 3], [4, 4], [5, 5]]), np.array([1, 1, 0, 0, 0]),
          validation_split = 0.1, epochs = 10)

Train on 4 samples, validate on 1 samples
Epoch 1/10
4/4 [==============================] - 1s 318ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 2/10
4/4 [==============================] - 0s 2ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 3/10
4/4 [==============================] - 0s 3ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 0s 2ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 0s 3ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 3ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.5000 - val_loss: 0.6931

In [28]:
model2.predict(np.array([[1, 2]]))

array([[0.5]], dtype=float32)

In [29]:
model1.evaluate(np.array([[1, 1]]), np.array([1]))

1/1 [==============================] - 0s 4ms/sample - loss: 0.6931 - accuracy: 0.0000e+00


[0.6931471824645996, 0.0]

#### custom model

In [30]:
class ResidualBlock(keras.layers.Layer):
    def __init__(self, n_layers, n_neurons, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(n_neurons, activation = 'relu',
                                          kernel_initializer = 'he_normal')
                      for _ in range(n_layers)]
    
    def call(self, inputs):
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)
        return inputs+Z

    def get_config(self):
        base_config = super().get_config()
        return {**base_config,
                'n_layers': len(self.hidden),
                'n_neurons': self.hidden[0].units}

In [31]:
obj = ResidualBlock(3, 5)

In [32]:
obj.get_config()

{'name': 'residual_block',
 'trainable': True,
 'dtype': 'float32',
 'n_layers': 3,
 'n_neurons': 5}

# Computing Gradients sing Autodiff

In [33]:
def func(w1, w2):
    return 3*w1**2+2*w1*w2

In [34]:
w1, w2 = 5, 3

eps = 1e-6

In [35]:
func(w1, w2)

105

In [36]:
(func(w1+eps, w2) - func(w1, w2))/eps

36.000003007075065

In [37]:
(func(w1, w2+eps) - func(w1, w2))/eps

10.000000003174137

In [38]:
def func(x, y, z):
    return (x**2+z**(1/2))/(5*y**2-z**3)

In [39]:
func(3, 2, 1)

0.5263157894736842

In [40]:
x, y, z = 3, 2, 1
eps = 1e-6

In [41]:
(func(x+eps, y, z)-func(x, y, z))/eps, (func(x, y, z)-func(x-eps, y, z))/eps

(0.3157895264171984, 0.31578942105703334)

In [42]:
(func(x, y+eps, z)-func(x, y, z))/eps

-0.5540161758244011

In [43]:
(func(x, y, z+eps)-func(x, y, z))/eps

0.10941837624844908

In [44]:
def func(w1, w2):
    return 3*w1**2+2*w1*w2

In [45]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)

In [46]:
with tf.GradientTape() as tape:
    z = func(w1, w2)

In [47]:
gradients = tape.gradient(z, [w1, w2])

In [48]:
gradients

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [49]:
with tf.GradientTape() as tape:
    z = func(w1, w2)

In [50]:
tape.gradient(z, w1)

<tf.Tensor: shape=(), dtype=float32, numpy=36.0>

In [51]:
# tape.gradient(z, w2)

In [52]:
with tf.GradientTape(persistent = True) as tape:
    z = func(w1, w2)

In [53]:
tape.gradient(z, w1)

<tf.Tensor: shape=(), dtype=float32, numpy=36.0>

In [54]:
tape.gradient(z, w2)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [55]:
del tape

In [56]:
m1 = tf.constant(5.)
m2 = tf.constant(3.)

with tf.GradientTape() as tape:
    tape.watch(m1)
    tape.watch(m2)
    
    z = func(m1, m2)

In [57]:
tape.gradient(z, m1)

<tf.Tensor: shape=(), dtype=float32, numpy=36.0>

In [58]:
# tape.gradient(z, m2)

In [59]:
with tf.GradientTape() as tape:
    tape.watch(m1)
    tape.watch(m2)
    
    z = func(m1, m2)

In [60]:
tape.gradient(z, [m1, m2])

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [61]:
with tf.GradientTape(persistent = True) as tape:
    tape.watch(m1)
    tape.watch(m2)
    
    z = func(m1, m2)

In [62]:
tape.gradient(z, m1)

<tf.Tensor: shape=(), dtype=float32, numpy=36.0>

In [63]:
tape.gradient(z, m2)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [64]:
with tf.GradientTape(persistent=True) as hessian_tape:
    hessian_tape.watch(m1)
    hessian_tape.watch(m2)
    
    with tf.GradientTape() as jacobian_tape:
        jacobian_tape.watch(m1)
        jacobian_tape.watch(m2)
    
        z = func(m1, m2)

    jacobians = jacobian_tape.gradient(z, [m1, m2])

    hessians = [hessian_tape.gradient(jacobian, [m1, m2])
            for jacobian in jacobians]

In [65]:
jacobians

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [66]:
hessians

[[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.0>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, None]]

In [67]:
with tf.GradientTape(persistent= True) as hessian_tape:
    with tf.GradientTape() as jacobian_tape:
        z = func(w1, w2)
    jacobians = jacobian_tape.gradient(z, [w1, w2])
hessians = [hessian_tape.gradient(jacobian, [w1, w2])
           for jacobian in jacobians]

In [68]:
jacobians

[<tf.Tensor: shape=(), dtype=float32, numpy=36.0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=10.0>]

In [69]:
hessians

[[<tf.Tensor: shape=(), dtype=float32, numpy=6.0>,
  <tf.Tensor: shape=(), dtype=float32, numpy=2.0>],
 [<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, None]]

#### Tensorflow graphs and functions

In [70]:
def func(x):
    m = 0
    for i in range(x):
        for j in range(x):
            for k in range(x):
                m += (i+j+k)
    return m

In [71]:
import time

In [72]:
func_tf = tf.function(func)

In [79]:
t11 = time.time()
print(func(100))
t12 = time.time()

t21 = time.time()
print(func_tf(100))
t22 = time.time()

148500000
tf.Tensor(148500000, shape=(), dtype=int32)


In [80]:
print(t12-t11)

0.18029284477233887


In [81]:
print(t22-t21)

0.0


## The Data API

In [82]:
X = tf.range(10)

In [83]:
X

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>

In [85]:
dataset = tf.data.Dataset.from_tensor_slices(X)

In [88]:
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [90]:
for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [91]:
dataset = tf.data.Dataset.range(10)

In [92]:
for i in dataset:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [93]:
X = tf.range(10)

In [107]:
dataset = tf.data.Dataset.from_tensor_slices(X)

In [108]:
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

In [109]:
dataset = dataset.repeat(3).batch(5)

In [110]:
for i in dataset:
    print(i)

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int32)
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int32)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int32)


In [111]:
dataset = tf.data.Dataset.from_tensor_slices(X)

In [112]:
dataset = dataset.repeat(3).batch(7, drop_remainder = True)

In [113]:
for i in dataset:
    print(i)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
